<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/logo-bdc.png" align="right" width="64"/>

# <span style="color:#336699">3º BIG TechTalks - Acesso, Visualização e Processamento de Imagens Sentinel-2 utilizando Python - Spatio Temporal Asset Catalog (STAC)</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Rennan F. B. Marujo<sup><a href="https://orcid.org/0000-0002-0082-9498"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup> e Gilberto R. Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Divisão de Observação da Terra e Geoinformática, Instituto Nacional de Pesquisas Espaciais (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contato: <a href="mailto:rennan.marujo@inpe.br">rennan.marujo@inpe.br</a>
    <br/><br/>
    Ultíma Atualização: 24 de Abril de 2025
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Resumo.</b> Este Jupyter Notebook é parte do 3º BIG TechTalks - Acesso, Visualização e Processamento de Imagens Sentinel-2 utilizando Python. Nos últimos anos, o padrão aberto SpatioTemporal Asset Catalog (STAC) tem sido utilizado por diversas iniciativas para indexar os grandes acervos de imagens de sensoriamento remoto. Desta forma, as coleções disponíveis nesses acervos podem ser consultadas e acessadas através de uma interface de programação de aplicações (API). Este Jupyter Notebook apresenta uma visão geral de como utilizar este serviço na linguagem Python para descoberta e acesso aos produtos de dados de sensoriamento remoto disponíveis no catálogo do INPE.
</div>

# Abrindo um shapefile e definindo uma area de estudo
<hr style="border:1px solid #0077b9;">

Neste exemplo vamos usar o arquivo `LEM_dataset_small.shp`. Para isso, será usada a biblioteca GeoPandas. Vamos abrir esse arquivo e extrair a informação de geometria.

In [ ]:
# !pip install folium

In [ ]:
# !pip install geopandas

In [ ]:
import folium
import geopandas as gpd

Podemos abrir um .shp diretamente ou um .zip contendo um .shp da seguinte forma:

In [ ]:
# zipfile = "LEM_dataset_small.zip"
# samples_df = gpd.read_file(zipfile)
# samples_df.head()

Ou podemos abrir um .shp armazenado online da seguinte forma:

In [ ]:
import io
import os
import requests
import tempfile
import zipfile

zipfile_url = "https://github.com/brazil-data-cube/code-gallery/raw/master/jupyter/Data/2025-sbsr/LEM_dataset_small.zip"
response = requests.get(zipfile_url)
with tempfile.TemporaryDirectory() as tmpdir:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(tmpdir)

        shp_file = [f for f in os.listdir(tmpdir) if f.endswith('.shp')][0]
        shp_path = os.path.join(tmpdir, shp_file)

        my_shp = gpd.read_file(shp_path)

        geometry_union = my_shp.geometry.union_all()
        bbox = geometry_union.bounds
        centroide = geometry_union.centroid
my_shp.head(3)

Vamos vizualizar onde encontra-se essa área:

In [ ]:
f = folium.Figure(width=1000, height=300)

centroide = my_shp.geometry.union_all().centroid
m = folium.Map(location=[centroide.y, centroide.x],
               zoom_start=11,
               tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
               attr="Esri World Imagery"
).add_to(f)

folium.GeoJson(my_shp).add_to(m)

m

Podemos inclusive visualizar o bbox dessa area:

In [ ]:
folium.Rectangle(
    bounds=[[bbox[1], bbox[0]], [bbox[3], bbox[2]]],  # [[min_lat, min_lon], [max_lat, max_lon]]
    color="#ff0000",
    fill=True,
    fill_opacity=0,
    tooltip=f"BBox: {bbox}"
).add_to(m)

m

# 1. Buscar imagens

---


Vamos começar buscando as imagens que utilizaremos.

Para isso utilizaremos o serviço SpatioTemporal Asset Catalog (STAC) por meio de um client na linguagem de programação Python.

<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/stac/stac.png?raw=true" align="right" width="66"/>

## Catálogo de Coleções de Imagem de Sensoriamento Remoto: **S**patio**T**emporal **A**sset **C**atalog (STAC)
<hr style="border:1px solid #0077b9;">

Boa parte dos produtos de imagem disponibilizados no [catálogo de imagens do INPE](https://data.inpe.br/stac/browser/?.language=en) são disponibilizados de maneira aberta na forma de arquivos otimizados para *cloud*, o denominado formato **C**loud **O**ptimized **G**eoTIFF (**COG**). Este formato permite que as aplicações possam utilizar as imagens através da Web com o melhor compromisso possível, incluindo o uso de pirâmide de multi-resolução para aplicações de visualização ou até mesmo a recuperação parcial de porções de uma imagem.


Esses produtos de dados podem ser consultados utilizando uma interface de programação de aplicações baseada no padrão aberto [**S**patio**T**emporal **A**sset **C**atalog (STAC)](https://stacspec.org/). Esta especificação, criada por organizações e especialistas do setor geoespacial, é baseada nos conceitos apresentados no diagrama abaixo:

<center>
<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/stac/stac-concept.png" width="480" />
<br/>
Modelo de funcionamento do STAC.
</center>

Em que:

- **Catalog**: É um tipo de objeto que fornece uma estrutura para vincular vários itens ou coleções STAC juntos ou mesmo outros catálogos. Na figura acima, o catálogo é composto de três coleções: Landsat/OLI, CBERS4/WFI e Sentinel-2/MSI.

- **Collection:** É uma especialização do catálogo que permite incluir informações adicionais sobre uma determinada coleção espaço-temporal. Uma coleção pode conter informações como o conjunto de bandas espectrais disponíveis das imagens, a extensão geográfica ou área de cobertura das imagens, o período de tempo que compreende a coleção, entre outras informações. Em geral, através da coleção chegamos aos itens dessa coleção.

- **Item**: Corresponde à unidade atômica de metadados, fornecendo *links* para os *assets* associados. Um *Item* é descrito através da notação GeoJSON, como uma feição (*feature*) contendo atributos específicos como a coleção a que ele pertence, propriedades temporais, *links* para os *assets* e coleções ou catálogos associados. Na figura acima, um `Item` equivale a uma cena obtida por um satélite em um determinado instante de tempo.

- **Asset**: Um *asset* é qualquer recurso geoespacial, como um arquivo de imagem ou arquivo vetorial, contendo informações sobre a supefície da Terra, em um determinado espaço e tempo.


A especificação conceitual do STAC permite dois tipos de implementações:

- **STAC estático:** Baseada em um conjunto de documentos JSON ligados que podem ser facilmente navegados. Ex: [CBERS na AWS](https://cbers-stac-1-0-0.s3.amazonaws.com/CBERS4/catalog.json).

- **STAC dinâmico:** Baseada em uma API RESTful, de modo que a navegação é realizada através de uma API de serviço web que permite realizar consultas utilizando uma linguagem padrão para acessar subconjuntos do catálogo. Ex: [BDC-STAC](https://data.inpe.br/bdc/stac/v1).


<br/>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Nota:</b> Como parte do aperfeiçoamento dos produtos e serviços disponibilizados pelo INPE à sociedade, encontra-se em desenvolvimento o novo portal <a href="https://data.inpe.br/">https://data.inpe.br/</a>, que faz parte da modernização da infraestrutura de serviços para acesso às imagens de satélites do acervo do instituto. Esse portal foi criado com o intuito de facilitar a pesquisa e obtenção das imagens disponibilizadas gratuitamente. Esse novo serviço tem como base as tecnologias desenvolvidas no projeto Brazil Data Cube, e está ancorado dentro do Programa Base de Informações Georreferenciadas (BIG) do INPE. Para navegar pelas coleções disponibilizadas no serviço STAC do INPE, utilize a instância do [STAC Browser](https://data.inpe.br/stac/browser/).
</div>

Cliente STAC no Python
<hr style="border:1px solid #0077b9;">

Para demonstrar o acesso aos produtos de dados do Brazil Data Cube, iremos utilizar uma bibloteca de software livre para Python denominada [PySTAC Client](https://pystac-client.readthedocs.io/en/stable/) (`pystac-client`).

Para instalar essa biblioteca no ambiente Jupyter, pode ser utilizado o comando `pip install`:

In [ ]:
# !pip install pystac-client

Uma vez instalada a biblioteca `pystac-client`, podemos carregar suas funcionalidades através do comando `import`, como mostrado abaixo:

In [ ]:
import pystac_client

Em geral, uma biblioteca do ecossistema Python possui uma constante especial para informar a versão da biblioteca carregada. Abaixo, apresentamos a versão carregada  da biblioteca `pystac-client`:

In [ ]:
pystac_client.__version__

<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/stac/stac-catalog.png?raw=true" align="right" width="300"/>

## Descobrindo as Coleções em um Catálogo STAC
<hr style="border:1px solid #0077b9;">

O endereço do serviço STAC do BDC é https://data.inpe.br/bdc/stac/v1/. Para descobir as coleções disponíveis no catálogo central desse serviço,
podemos utilizar a classe `Client` do pacote `pystac_client`.

Essa classe possui um método denominado `open` que permite informar a URL do serviço STAC a ser utilizado. Assim que chamado, esse método realiza uma consulta ao serviço STAC, recuperando as informações do catálogo central.

In [ ]:
catalogo = pystac_client.Client.open('https://data.inpe.br/bdc/stac/v1/')
catalogo

O objeto retornado possui atributos como `id`, `title` e `description`:

In [ ]:
catalogo.id

In [ ]:
catalogo.title

In [ ]:
catalogo.description

Um catálogo também contém propriedades como:

- `links`: Lista de endereços para todas as coleções disponíveis no catálogo.

- `conformsTo`: Lista das capacidades do serviço. No caso acima, a classe de conformidade `item-search` especifica que o serviço do BDC é capaz de realizar a busca de itens percorrendo todas as coleções.

O método `get_collections` permite iterar por todas as coleções existentes no catálogo.

In [ ]:
for colecao in catalogo.get_collections():
    print(f"{colecao.id}: {colecao.title}", end="\n"*2)

In [ ]:
sentinel2 = catalogo.get_collection("S2_L2A-1")
sentinel2

Na saída acima, destaca-se os seguintes metadados:

- Identificador, título e descrição da coleção, nas chaves `id`, `title` e `description`, respectivamente.

- A cobertura espacial das imagens dessa coleção, na chave `extent -> spatial -> bbox`.

- A disponibilidade temporal de imagens, na chave `extent -> temporal -> interval`.

- As bandas disponíveis na coleção, na chave `properties -> eo:bands`

<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/stac/stac-catalog.png?raw=true" align="right" width="300"/>

### Recuperando os Items de uma Coleção
<hr style="border:1px solid #0077b9;">

O método `get_items` permite atravessar o conjunto de itens de uma coleção. O trecho de código abaixo mostra como percorrer os 20 primeiros itens da coleção `sentinel2`:

In [ ]:
import itertools

for item in itertools.islice(sentinel2.get_items(), 20):
    print(f"{item.id}")

Um *item* possui propriedades como:

- Identificador do item dentro da coleção, que pode ser obtido na chave `id`.

- O *footprint* da imagem, na chave `geometry`.

- Retângulo envolvente da cena, na chave `bbox`.

- Propriedades como porcentagem de cobertura de nuvem na cena, na chave `properties -> eo:cloud_cover`, e a data associada com a passagem da imagem, chave `properties -> datetime`.

- O conjunto de *assets*, isto é, dos arquivos que compõem de fato o *item*. Nessa chave teremos

A célula de código abaixo irá apresentar todas as propriedades do último *item* acessado no código anterior:

### Selecionando imagens por região de interesse e intervalo de datas
<hr style="border:1px solid #0077b9;">

Embora o método `get_items` permita recuperar (ou atravessar) todos os itens de uma coleção, esse método não é muito útil na prática pois quase sempre desejamos pesquisar imagens em um acervo utilizando algum tipo de critério para selecionar essas imagens, como um certo período, uma certa regiao de interesse e o limite de cobertura de nuvem aceitável.

O método `search` de uma catálogo pode ser utilizado para realizar uma busca mais refinada. Os principais parâmetros desse método são:

- `collections`: Lista com o nome de uma ou mais coleções às quais a busca será limitada. Também podemos passar objetos do tipo coleção. Se omitido esse parâmetro, todas as coleções serão consideradas.

- `bbox`: Retângulo de interesse da busca. (Parâmetro opcional)

- `datetime`: Podemos utilizar uma data específica ou um intervalo de datas. Essas datas devem ser expressas de acordo com a [RFC-3339](https://datatracker.ietf.org/doc/html/rfc3339). (Parâmetro opcional)

- `limit`: Recomendação passada ao serviço para que ele use este número como a quantidade de itens na paginação dos resultados. (Parâmetro opcional)

- `intersects`: Uma geometria usada para definir a região de interesse. Deve ser representada como um GeoJSON na forma de uma string ou dicionário ou um objeto que implemente a propriedade `__geo_interface__`. (Parâmetro opcional)

A célula de código abaixo mostra como selecionar todos os itens da coleção `sentinel2` para uma dada região e período de tempo, com uma sugestão de paginação de 100 itens e buscando por imagens com cobertura de nuvens menor do que 10%.

In [ ]:
item_search = catalogo.search(
    collections=[sentinel2],
    bbox=bbox,
    datetime='2024-01-01/2025-03-21',
    query = {
      "eo:cloud_cover": {
          "lt" : 10
      }
    },
    limit = 100
)

O número de itens encontrados pode ser verificado através do método `matched()`:

In [ ]:
item_search.matched()

Vamos fazer essa mesma busca sem considerar o percentual de nuvens, para obter todas as imagens disponíveis para a nossa area:

In [ ]:
item_search = catalogo.search(
    collections=[sentinel2],
    bbox=bbox,
    datetime='2024-01-01/2025-03-21',
    limit = 100
)

Podemos observar que temos um total maior de imagens:

In [ ]:
item_search.matched()

Para atravesar o conjunto de itens retornados pela busca, podemos utilizar o método `items()`.

In [ ]:
for i, item in enumerate(item_search.items()):
    print(i, item.id, sep='\t')

Outra opção é utilizar a estrutura de dados listas para essa verificação:

In [ ]:
items_list = list(item_search.items())
items_list

<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/stac/stac-asset.png?raw=true" align="right" width="300"/>

Assets
<hr style="border:1px solid #0077b9;">

A partir de um *item*, podemos recuperar todos os *assets* associados. Os *assets* basicamente trazem a informação da URL onde o arquivo associado encontra-se.

Partindo dos itens da coleção `sentinel2` selecionados anteriormente, vamos construir um objeto do tipo `FeatureCollection` contendo todos esses itens.

In [ ]:
items = item_search.item_collection()
items

Tomando o primeiro item dessa coleção como referência:

In [ ]:
item = items[0]
item

podemos observar que a chave `assets` contém informações que nos levam de fato ao arquivo de imagem das bandas espectrais e de metadados:

In [ ]:
item.assets

Vamos acessar o *asset* associado à banda do vermelho no visível:

In [ ]:
B04 = item.assets['B04']
B04

In [ ]:
B04.href

# 2. Visualizar (Cor verdadeira e Falsa Cor)

---


Para recuperar a matrix de pixels da imagem indicada no atributo `href` do *asset* será utilizada a biblioteca `rasterio`.

In [ ]:
# !pip install rasterio

Vamos fazer a visualização desse dado.

OBS: Pode demorar um pouco a depender da conexão com a internet, pois estamos obtendo uma banda inteira da imagem.

In [ ]:
import rasterio
from matplotlib import pyplot as plt

with rasterio.open(B04.href) as src:
    band = src.read(1)

plt.imshow(band, cmap='gray')
plt.show()

Outra busca que podemos realizar no catálogo consiste em usar o `intersects`:

In [ ]:
item_search = catalogo.search(
    collections = [sentinel2],
    intersects = geometry_union,
    datetime = '2024-01-01/2025-03-21',
    limit = 100
)

In [ ]:
item_search.matched()

De posse do resultado da pesquisa, podemos gerar uma coleção de feições para facilitar a manipulação do resultado:

In [ ]:
items = item_search.item_collection()
items

## Recuperando parte de uma imagem correspondente a uma região
<hr style="border:1px solid #0077b9;">

Vamos definir algumas funções auxiliares para nos ajudar nesse Jupyter Notebook.

- `read_img`: Lê uma imagem usando Window.

- `normalize`: Normaliza, para visualização, o valor de imagens.


In [ ]:
import numpy as np
from rasterio.crs import CRS
from rasterio.warp import transform
from rasterio.windows import from_bounds

def read(uri: str, bbox: list, masked: bool = True, crs: str = None):
    """Read raster window as numpy.ma.masked_array."""
    source_crs = CRS.from_string('EPSG:4326')
    if crs:
        source_crs = CRS.from_string(crs)

    # Expects the bounding box has 4 values
    w, s, e, n = bbox

    with rasterio.open(uri) as dataset:
        transformer = transform(source_crs, dataset.crs, [w, e], [s, n])
        window = from_bounds(transformer[0][0], transformer[1][0],
                             transformer[0][1], transformer[1][1], dataset.transform)
        return dataset.read(1, window=window, masked=masked)

def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

In [ ]:
assets = items[0].assets
assets

Vamos armazenar a url dos assets das bandas azul, verde, vermelho e infra-vermelho próximo.

In [ ]:
blue_asset = assets['B02']
green_asset = assets['B03']
red_asset = assets['B04']
nir_asset = assets['B08']

blue_asset.href

Agora podemos usar o *Window* para abrir somente um pedaço da imagem

In [ ]:
b02_image = read(items[7].assets['B02'].href, bbox=bbox)
b03_image = read(items[7].assets['B03'].href, bbox=bbox)
b04_image = read(items[7].assets['B04'].href, bbox=bbox)
b08_image = read(items[7].assets['B08'].href, bbox=bbox)

Assim podemos visualizar as 4 imagens carregadas:

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(12, 4))
ax1.imshow(b02_image, cmap='gray')
ax1.set_title("Banda 2 (Azul)")
ax2.imshow(b03_image, cmap='gray')
ax2.set_title("Banda 3 (Verde)")
ax3.imshow(b04_image, cmap='gray')
ax3.set_title("Banda 4 (Vermelho)")
ax4.imshow(b08_image, cmap='gray')
ax4.set_title("Banda 8 (Infra-vermelho Próximo)")

Podemos fazer um stack e visualizar a *Cor Verdadeira*:

In [ ]:
rgb_normalized_stack = np.dstack((normalize(b04_image), normalize(b03_image), normalize(b02_image)))
plt.imshow(rgb_normalized_stack)

Bem como compor uma *Falsa Cor*

In [ ]:
rgb_normalized_stack = np.dstack((normalize(b08_image), normalize(b04_image), normalize(b03_image)))
plt.imshow(rgb_normalized_stack)